# 強化学習の実験に便利なコード

タイトルの通りです。よく使う関数をまとめます。

## 無限ホライゾン

### マルコフ決定過程の生成

[強化学習の青本](https://amzn.asia/d/2epmlxT)に従ったMDPの定義用のコードです。
MDPを次で定義します。

1. 有限状態集合: $S=\{1, \dots, |S|\}$
2. 有限行動集合: $A=\{1, \dots, |A|\}$
3. 遷移確率行列: $P\in \mathbb{R}^{SA\times S}$
4. 報酬行列: $r\in \mathbb{R}^{S\times A}$
5. 割引率: $\gamma \in [0, 1)$
6. 初期状態: $\mu \in \mathbb{R}^{S}$

In [1]:
import numpy as np
import jax.numpy as jnp
from jax.random import PRNGKey
import jax
from typing import NamedTuple, Optional

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
gamma = 0.8  # 割引率


# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(S, A))
assert rew.shape == (S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 初期状態分布を適当に作ります
key, _ = jax.random.split(key)
mu = jax.random.uniform(key, shape=(S,))
mu = mu / jnp.sum(mu)
np.testing.assert_allclose(mu.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    gamma: float  # 割引率
    H: int  # エフェクティブホライゾン
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列
    mu: jnp.array  # 初期分布
    optimal_Q: Optional[jnp.ndarray] = None  # 最適Q値

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


H = int(1 / (1 - gamma))
mdp = MDP(S_set, A_set, gamma, H, rew, P, mu)

print("状態数：", mdp.S)
print("行動数：", mdp.A)
print("割引率：", mdp.gamma)
print("ホライゾン：", mdp.H)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


状態数： 10
行動数： 3
割引率： 0.8
ホライゾン： 5


### 動的計画法

参考

* [Safe Policy Iteration](http://proceedings.mlr.press/v28/pirotta13.pdf)の２ページ目

**表記**

* 内積の記法: $f_1, f_2 \in \mathbb{R}^{S\times A}$に対して、$\langle f_1, f_2 \rangle = (\sum_{a\in A} f_1(s, a)f_2(s, a))_s \in \mathbb{R}^S$とします。これは方策についての和を省略するときなどに便利です。例えば$\langle \pi, q_\pi\rangle = v_\pi$です。
* 方策行列（$\Pi^\pi \in \mathbb{R}^{S\times SA}$）：$\langle \pi, q\rangle$を行列で書きたいときに便利。
    * $\Pi^\pi(s,(s, a))=\pi(a \mid s)$ 
    * $\Pi^\pi q^\pi = \langle \pi, q^\pi \rangle = v^\pi$が成立。
* 遷移確率行列１（$P^\pi \in \mathbb{R}^{SA\times SA}$）: 次の状態についての方策の情報を追加したやつ。
    * $P^\pi = P \Pi^\pi$
    * Q値を使ったベルマン期待作用素とかで便利。$q^\pi = r + \gamma P^\pi q^\pi$が成立。
    * $(I - \gamma P^\pi)^{-1}r = q^\pi$が成立する。
* 遷移確率行列２（$\bar{P}^\pi \in \mathbb{R}^{S\times S}$）: 方策$\pi$のもとでの状態遷移の行列。
    * $\bar{P}^\pi = \Pi^\pi P$
    * V値を使ったベルマン期待作用素とかで便利。$v^\pi = \Pi^\pi r + \gamma \bar{P}^\pi v^\pi$。
    * $(I - \gamma \bar{P}^\pi)^{-1}\Pi^\pi r = v^\pi$が成立する。
* 割引訪問頻度１（$d^\pi_\mu \in \mathbb{R}^{SA}$）：S, Aについての割引累積訪問頻度
    * ${d}^\pi_\mu (s, a) = \pi(a|s) \sum_{s_0} \mu(s_0) \sum_{t=0}^\infty \mathrm{Pr}\left(S_t=s|S_0=s_0, M(\pi)\right)$がで定義される。
    * $d^\pi_\mu = \mu \Pi^\pi (I - \gamma P^\pi)^{-1} = \mu (I - \gamma \bar{P}^\pi)^{-1} \Pi^\pi$が成立。
    * $d^\pi_\mu = \mu \Pi^\pi + \gamma d^\pi_\mu P^\pi$が成立。動的計画法のように解ける。
* 割引訪問頻度２（$\bar{d}^\pi_\mu \in \mathbb{R}^{S}$）：Sについての割引累積訪問頻度
    * $\bar{d}^\pi_\mu (s) = \sum_{s_0} \mu(s_0) \sum_{t=0}^\infty \mathrm{Pr}\left(S_t=s|S_0=s_0, M(\pi)\right)$で定義される。
    * $\bar{d}^\pi_\mu = \mu (I - \gamma \bar{P}^\pi)^{-1}$が成立。
    * $\bar{d}^\pi_\mu = \mu + \gamma \bar{d}^\pi_\mu \bar{P}^\pi$が成立。動的計画法のように解ける。


**実装した関数**

* ``compute_greedy_policy``: Q関数 ($S \times A \to \mathcal{R}$) の貪欲方策を返します
* ``compute_optimal_Q``: MDPの最適Q関数 $q_* : S \times A \to \mathcal{R}$ を返します。
* ``compute_policy_Q``: 方策 $\pi$ のQ関数 $q_\pi : S \times A \to \mathcal{R}$ を返します。
* ``compute_policy_matrix``: 方策$\pi$の行列${\Pi}^{\pi}$を返します。
* ``compute_policy_visit_sa``: 方策 $\pi$ の割引訪問頻度１${d}^\pi_\mu \in \mathbb{R}^{S\times A}$ を返します。
* ``compute_policy_visit_s``: 方策 $\pi$ の割引訪問頻度２$\bar{d}^\pi_\mu \in \mathbb{R}^{S}$ を返します。

In [18]:
import jax
import jax.numpy as jnp
import numpy as np
from functools import partial
import chex


@jax.jit
def compute_greedy_policy(Q: jnp.ndarray):
    """Q関数の貪欲方策を返します

    Args:
        Q (jnp.ndarray): (SxA)の行列

    Returns:
        greedy_policy (jnp.ndarray): (SxA)の行列
    """
    greedy_policy = jnp.zeros_like(Q)
    S, A = Q.shape
    greedy_policy = greedy_policy.at[jnp.arange(S), Q.argmax(axis=1)].set(1)
    assert greedy_policy.shape == (S, A)
    return greedy_policy


@partial(jax.jit, static_argnames=("S", "A"))
def _compute_optimal_Q(mdp: MDP, S: int, A: int):
    """MDPについて、ベルマン最適作用素を複数回走らせて最適価値関数を動的計画法で計算します。
    Args:
        mdp (MDP)

    Returns:
        optimal_Q (jnp.ndarray): (SxA)の行列
    """

    def backup(optimal_Q):
        next_v = mdp.P @ optimal_Q.max(axis=1)
        assert next_v.shape == (S, A)
        return mdp.rew + mdp.gamma * next_v
    
    optimal_Q = jnp.zeros((S, A))
    body_fn = lambda i, Q: backup(Q)
    return jax.lax.fori_loop(0, mdp.H + 100, body_fn, optimal_Q)

compute_optimal_Q = lambda mdp: _compute_optimal_Q(mdp, mdp.S, mdp.A)


@jax.jit
def compute_policy_Q(mdp: MDP, policy: jnp.ndarray):
    """MDPと方策について、ベルマン期待作用素を複数回走らせて価値関数を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (jnp.ndarray): (SxA)の行列

    Returns:
        optimal_Q (jnp.ndarray): (SxA)の行列
    """
    S, A = policy.shape
    chex.assert_shape(policy, (mdp.S, mdp.A))

    def backup(policy_Q):
        max_Q = (policy * policy_Q).sum(axis=1)
        next_v = mdp.P @ max_Q
        assert next_v.shape == (S, A)
        return mdp.rew + mdp.gamma * next_v
    
    policy_Q = jnp.zeros((S, A))
    body_fn = lambda i, Q: backup(Q)
    return jax.lax.fori_loop(0, mdp.H + 100, body_fn, policy_Q)


@jax.jit
def compute_policy_matrix(policy: jnp.ndarray):
    """
    上で定義した方策行列を計算します。方策についての内積が取りたいときに便利です。
    Args:
        policy (jnp.ndarray): (SxA)の行列

    Returns:
        policy_matrix (jnp.ndarray): (SxSA)の行列
    """
    S, A = policy.shape
    PI = policy.reshape(1, S, A)
    PI = jnp.tile(PI, (S, 1, 1))
    eyes = jnp.eye(S).reshape(S, S, 1)
    PI = (eyes * PI).reshape(S, S*A)
    return PI


@jax.jit
def compute_policy_visit_sa(mdp: MDP, policy: jnp.ndarray, init_dist: jnp.ndarray):
    """MDPと方策について、割引訪問頻度１を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (jnp.ndarray): (SxA)の行列
        init_dist (jnp.ndarray): (S) 初期状態の分布

    Returns:
        visit (jnp.ndarray): (SxA)の行列
    """
    S, A = policy.shape
    chex.assert_shape(policy, (mdp.S, mdp.A))
    Pi = compute_policy_matrix(policy)
    PPi = mdp.P.reshape(S*A, S) @ Pi 

    def backup(visit):
        next_visit = mdp.gamma * visit @ PPi
        return init_dist @ Pi + next_visit
    
    body_fn = lambda i, visit: backup(visit)
    visit = jnp.zeros(S*A)
    visit = jax.lax.fori_loop(0, mdp.H + 100, body_fn, visit)
    visit = visit.reshape(S, A)
    return visit


@jax.jit
def compute_policy_visit_s(mdp: MDP, policy: jnp.ndarray, init_dist: jnp.ndarray):
    """MDPと方策について、割引訪問頻度２を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (jnp.ndarray): (SxA)の行列
        init_dist (jnp.ndarray): (S) 初期状態の分布

    Returns:
        visit (jnp.ndarray): (S)のベクトル
    """
    S, A = policy.shape
    chex.assert_shape(policy, (mdp.S, mdp.A))
    Pi = compute_policy_matrix(policy)
    PiP = Pi @ mdp.P.reshape(S*A, S) 

    def backup(visit):
        next_visit = mdp.gamma * visit @ PiP
        return init_dist + next_visit
    
    body_fn = lambda i, visit: backup(visit)
    visit = jnp.zeros(S)
    visit = jax.lax.fori_loop(0, mdp.H + 100, body_fn, visit)
    return visit


# 動的計画法による最適価値関数
optimal_Q_DP = compute_optimal_Q(mdp)
optimal_V_DP = optimal_Q_DP.max(axis=1)
optimal_policy = compute_greedy_policy(optimal_Q_DP)
mdp = mdp._replace(optimal_Q=optimal_Q_DP)


# 逆行列による解法 Q
Pi = compute_policy_matrix(optimal_policy)
PPi = mdp.P.reshape(S*A, S) @ Pi
optimal_Q_inv = jnp.linalg.inv(jnp.eye(S*A) - mdp.gamma * PPi) @ mdp.rew.reshape(S*A)
print("Qベースの動的計画法と逆行列の解の差：", jnp.abs(optimal_Q_inv - optimal_Q_DP.reshape(-1)).max())


# 逆行列による解法 V
Pi = compute_policy_matrix(optimal_policy)
PiP = Pi @ mdp.P.reshape(S*A, S) 
Pirew = Pi @ mdp.rew.reshape(S*A)
optimal_V_inv = jnp.linalg.inv(jnp.eye(S) - mdp.gamma * PiP) @ Pirew
print("Vベースの動的計画法と逆行列の解の差：", jnp.abs(optimal_V_inv - optimal_V_DP.reshape(-1)).max())


# 割引訪問頻度の計算１
d_pi_DP = compute_policy_visit_sa(mdp, optimal_policy, mdp.mu).reshape(-1)
d_pi_inv = (mdp.mu @ jnp.linalg.inv(jnp.eye(S) - mdp.gamma * PiP) @ Pi)
print("動的計画法で計算した割引訪問頻度と逆行列の解の差", jnp.abs(d_pi_DP - d_pi_inv).max())
optimal_return_DP = mdp.mu @ optimal_V_DP
optimal_return_visit = Pi @ d_pi_inv @ Pirew
print("割引訪問頻度で計算した期待リターンと動的計画法の解の差", jnp.abs(optimal_return_DP - optimal_return_visit).max())



# 割引訪問頻度の計算２
d_pi_DP = compute_policy_visit_s(mdp, optimal_policy, mdp.mu)
d_pi_inv = mdp.mu @ jnp.linalg.inv(jnp.eye(S) - mdp.gamma * PiP)
print("動的計画法で計算した割引訪問頻度と逆行列の解の差", jnp.abs(d_pi_DP - d_pi_inv).max())
optimal_return_DP = mdp.mu @ optimal_V_DP
optimal_return_visit = d_pi_inv @ Pirew
print("割引訪問頻度で計算した期待リターンと動的計画法の解の差", jnp.abs(optimal_return_DP - optimal_return_visit).max())


d_pi_inv_SA1 = mdp.mu @ jnp.linalg.inv(jnp.eye(S) - mdp.gamma * PiP) @ Pi
d_pi_inv_SA2 = mdp.mu @ Pi @ jnp.linalg.inv(jnp.eye(S*A) - mdp.gamma * PPi)

print("SAについての割引訪問頻度の求め方２つの差：", jnp.abs(d_pi_inv_SA1 - d_pi_inv_SA2).max())

Qベースの動的計画法と逆行列の解の差： 4.7683716e-07
Vベースの動的計画法と逆行列の解の差： 4.7683716e-07
動的計画法で計算した割引訪問頻度と逆行列の解の差 1.1920929e-07
割引訪問頻度で計算した期待リターンと動的計画法の解の差 0.0
動的計画法で計算した割引訪問頻度と逆行列の解の差 1.1920929e-07
割引訪問頻度で計算した期待リターンと動的計画法の解の差 0.0
SAについての割引訪問頻度の求め方２つの差： 5.9604645e-08


## 強化学習用

**実装した関数**

* ``sample_next_state``: 状態・行動の集合$D$のそれぞれについて次状態を$N$個返します。訪問した(状態, 行動, 次状態)のカウントも返します。
* ``collect_samples_eps_greedy``: $q\in \mathbb{R}^{S\times A}$のε-貪欲方策で$N$回インタラクションしてサンプルを集めます。訪問した(状態, 行動, 次状態)のカウントも返します。

In [3]:
from jax.random import PRNGKey


@partial(jax.jit, static_argnames=("N",))
def sample_next_state(mdp: MDP, N: int, key: PRNGKey, D: jnp.array):
    """ 遷移行列Pに従って次の状態をN個サンプルします
    Args:
        mdp (MDP)
        N (int): サンプルする個数
        key (PRNGKey)
        D (jnp.ndarray): 状態行動対の集合 [(s1, a1), (s2, a2), ...]

    Returns:
        new_key (PRNGKey)
        next_s_set (jnp.ndarray): (len(D) x N) の次状態の集合
        count_SAS (jnp.ndarray): 各(状態, 行動, 次状態)のペアの出現回数を格納した(S x A x S) の行列
    """

    # 次状態をサンプルします
    new_key, key = jax.random.split(key)
    keys = jax.random.split(key, num=len(D))
    @jax.vmap
    def choice(key, sa):
        return jax.random.choice(key, mdp.S_set, shape=(N,), p=P[sa[0], sa[1]])
    next_s = choice(keys, D)

    # 集めたサンプルについて、(s, a, ns)が何個出たかカウントします。
    S, A, S = mdp.P.shape
    count_SAS = jnp.zeros((S*A, S))
    count_D_next_S = jax.vmap(lambda next_s: jnp.bincount(next_s, minlength=S, length=S))(next_s)
    D_ravel = jnp.ravel_multi_index(D.T, (S, A), mode="wrap")
    count_SAS = count_SAS.at[D_ravel].add(count_D_next_S)
    return new_key, next_s, count_SAS


key = jax.random.PRNGKey(0)
N = 20000
D = jnp.array([(1, 2), (2, 1), (0, 0), (3, 1), (0, 0)])
key, next_states, count_SAS = sample_next_state(mdp, N, key, D)
assert count_SAS.sum() == N * len(D)
assert next_states.shape == (len(D), N)
s, a = D[0]
P0_approx = jnp.bincount(next_states[0], minlength=S) / N
np.testing.assert_allclose(P0_approx, mdp.P[s, a], atol=1e-2)


@partial(jax.jit, static_argnames=("N",))
def collect_samples_eps_greedy(mdp: MDP, N: int, key: PRNGKey, q: jnp.array, init_s: int, epsilon: float=0.0):
    """ MDPとインタラクションしてサンプルをN個集めます。qの貪欲方策に従って動きます。
    Args:
        mdp (MDP)
        N (int): サンプルする個数
        key (PRNGKey)
        q (jnp.ndarray): 行動価値関数
        init_s (int): 初期状態
        epsilon (float): ε-貪欲のパラメータ

    Returns:
        new_key (PRNGKey)
        sars (jnp.ndarray): (状態, 行動, 報酬, 次状態) x N の軌跡
        count_SAS (jnp.ndarray): 各(状態, 行動, 次状態)のペアの出現回数を格納した(S x A x S) の行列
    """
    chex.assert_shape(q, (mdp.S, mdp.A))
    S, A = q.shape

    def body_fn(n, args):
        key, sars, s, count_SAS = args

        # ε-貪欲方策を実行します
        a = q[s].argmax()
        key, key1, key2 = jax.random.split(key, num=3)
        random_a = jax.random.choice(key1, A)
        a = jnp.where(jax.random.uniform(key2) > epsilon, a, random_a)
        
        # 次状態をサンプルします
        key, key1 = jax.random.split(key)
        next_s = jax.random.choice(key1, mdp.S_set, p=P[s, a])

        # 集めたデータを記録します
        r = mdp.rew[s, a]
        sars = sars.at[n].set((s, a, r, next_s))
        count_SAS = count_SAS.at[s, a, next_s].add(1)
        return key, sars, next_s, count_SAS

    sars = jnp.zeros((N, 4))
    count_SAS = jnp.zeros((S, A, S))
    args = key, sars, init_s, count_SAS
    key, sars, next_s, count_SAS = jax.lax.fori_loop(0, N, body_fn, args)
    return key, sars, next_s, count_SAS


key, sars, next_s, count_SAS = collect_samples_eps_greedy(mdp, N, key, mdp.optimal_Q, 0)
assert sars.shape == (N, 4)
assert count_SAS.sum() == N

## 有限ホライゾン

### マルコフ決定過程の生成

**参考**

有限MDPの定義については[Reinforcement Learning: Theory and Algorithms](https://rltheorybook.github.io/)の1.2章を参照しています。
有限ホライゾンの場合、遷移行列や報酬関数が各ステップで変わる設定を考えます。

1. 有限状態集合: $S=\{1, \dots, |S|\}$
2. 有限行動集合: $A=\{1, \dots, |A|\}$
3. $h$ステップ目の遷移確率行列: $P_h\in \mathbb{R}^{SA\times S}$
4. $h$ステップ目の報酬行列: $r_h\in \mathbb{R}^{S\times A}$
5. ホライゾン: $H$
6. 初期状態: $\mu \in \mathbb{R}^{S}$

In [4]:
import jax.numpy as jnp
import numpy as np
from typing import NamedTuple, Optional
from jax.random import PRNGKey

key = PRNGKey(0)

S = 10  # 状態集合のサイズ
A = 3  # 行動集合のサイズ
S_set = jnp.arange(S)  # 状態集合
A_set = jnp.arange(A)  # 行動集合
H = 5  # ホライゾン

# 報酬行列を適当に作ります
key, _ = jax.random.split(key)
rew = jax.random.uniform(key=key, shape=(H, S, A))
assert rew.shape == (H, S, A)


# 遷移確率行列を適当に作ります
key, _ = jax.random.split(key)
P = jax.random.uniform(key=key, shape=(H, S*A, S))
P = P / jnp.sum(P, axis=-1, keepdims=True)  # 正規化して確率にします
P = P.reshape(H, S, A, S)
np.testing.assert_allclose(P.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 初期状態分布を適当に作ります
key, _ = jax.random.split(key)
mu = jax.random.uniform(key, shape=(S,))
mu = mu / jnp.sum(mu)
np.testing.assert_allclose(mu.sum(axis=-1), 1, atol=1e-6)  # ちゃんと確率行列になっているか確認します


# 状態集合, 行動集合, 割引率, 報酬行列, 遷移確率行列が準備できたのでMDPのクラスを作ります

class MDP(NamedTuple):
    S_set: jnp.array  # 状態集合
    A_set: jnp.array  # 行動集合
    H: int  # ホライゾン
    rew: jnp.array  # 報酬行列
    P: jnp.array  # 遷移確率行列
    mu: jnp.array  # 初期分布
    optimal_Q: Optional[jnp.ndarray] = None  # 最適Q値

    @property
    def S(self) -> int:  # 状態空間のサイズ
        return len(self.S_set)

    @property
    def A(self) -> int:  # 行動空間のサイズ
        return len(self.A_set)


mdp = MDP(S_set, A_set, H, rew, P, mu)

print("状態数：", mdp.S)
print("行動数：", mdp.A)
print("ホライゾン：", mdp.H)

状態数： 10
行動数： 3
ホライゾン： 5


### 動的計画法

**表記**

* ステップ$h$の方策行列（$\Pi_h^\pi \in \mathbb{R}^{S\times SA}$）：$\langle \pi_h, q\rangle$を行列で書きたいときに便利。
    * $\Pi_h^\pi(s,(s, a))=\pi_h(a \mid s)$ 
    * $\Pi_h^\pi q_h^\pi = \langle \pi, q_h^\pi \rangle = v_h^\pi$が成立。
* ステップ$h$の遷移確率行列１（$P_h^\pi \in \mathbb{R}^{SA\times SA}$）: 次の状態についての方策の情報を追加したやつ。
    * $P_h^\pi = P_h \Pi_h^\pi$
    * Q値を使ったベルマン期待作用素とかで便利。$q_h^\pi = r_h + P_h^\pi q^\pi$が成立。
* ステップ$h$の遷移確率行列２（$\bar{P}_h^\pi \in \mathbb{R}^{S\times S}$）: 方策$\pi$のもとでの状態遷移の行列。
    * $\bar{P}_h^\pi = \Pi_h^\pi P_h$
    * V値を使ったベルマン期待作用素とかで便利。$v_h^\pi = \Pi^\pi r_h + \gamma \bar{P}_h^\pi v^\pi$。
* ステップ$h$の割引訪問頻度（$d^\pi_{h,\mu} \in \mathbb{R}^{SA}$）：S, Aについての割引累積訪問頻度
    * ${d}^\pi_{h,\mu} (s, a) = \pi(a|s) \sum_{s_0} \mu(s_0) \sum_{t=0}^h \mathrm{Pr}\left(S_t=s|S_0=s_0, M(\pi)\right)$


**実装した関数**

* ``compute_greedy_policy``: Q関数 ($H\times S \times A \to \mathcal{R}$) の貪欲方策を返します
* ``compute_optimal_Q``: MDPの最適Q関数 $q_* : H\times S \times A \to \mathcal{R}$ を返します。
* ``compute_policy_Q``: 方策 $\pi$ のQ関数 $q_\pi : H\times S \times A \to \mathcal{R}$ を返します。
* ``compute_policy_matrix``: 方策$\pi$の行列${\Pi}^{\pi} : H \times S \times SA$を返します。
* ``compute_policy_visit``: 方策 $\pi$ の割引訪問頻度${d}^\pi_{\mu} : {H\times S \times A}$ を返します。

In [8]:
from functools import partial
import jax
import chex


@jax.jit
def compute_greedy_policy(Q: jnp.ndarray):
    """Q関数の貪欲方策を返します

    Args:
        Q (jnp.ndarray): (HxSxA)の行列

    Returns:
        greedy_policy (jnp.ndarray): (HxSxA)の行列
    """
    greedy_policy = jnp.zeros_like(Q)
    H, S, A = Q.shape
    
    def body_fn(i, greedy_policy):
        greedy_policy = greedy_policy.at[i, jnp.arange(S), Q[i].argmax(axis=-1)].set(1)
        return greedy_policy

    greedy_policy = jax.lax.fori_loop(0, H, body_fn, greedy_policy)
    chex.assert_shape(greedy_policy, (H, S, A))
    return greedy_policy


@partial(jax.jit, static_argnames=("H", "S", "A"))
def _compute_optimal_Q(mdp: MDP, H: int, S: int, A: int):
    """ベルマン最適作用素をホライゾン回走らせて最適価値関数を動的計画法で計算します。
    Args:
        mdp (MDP)

    Returns:
        optimal_Q (jnp.ndarray): (HxSxA)の行列
    """

    def backup(i, optimal_Q):
        h = H - i - 1
        max_Q = optimal_Q[h+1].max(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        optimal_Q = optimal_Q.at[h].set(mdp.rew[h] + next_v)
        return optimal_Q
    
    optimal_Q = jnp.zeros((H+1, S, A))
    optimal_Q = jax.lax.fori_loop(0, mdp.H, backup, optimal_Q)
    return optimal_Q[:-1]

compute_optimal_Q = lambda mdp: _compute_optimal_Q(mdp, mdp.H, mdp.S, mdp.A)


@jax.jit
def compute_policy_Q(mdp: MDP, policy: jnp.ndarray):
    """ベルマン期待作用素をホライゾン回走らせて価値関数を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (np.ndarray): (HxSxA)の行列

    Returns:
        optimal_Q (jnp.ndarray): (HxSxA)の行列
    """
    H, S, A = policy.shape

    def backup(i, policy_Q):
        h = H - i - 1
        max_Q = (policy[h+1] * policy_Q[h+1]).sum(axis=1)
        next_v = mdp.P[h] @ max_Q
        chex.assert_shape(next_v, (S, A))
        policy_Q = policy_Q.at[h].set(mdp.rew[h] + next_v)
        return policy_Q
    
    policy_Q = jnp.zeros((H+1, S, A))
    policy_Q = jax.lax.fori_loop(0, mdp.H, backup, policy_Q)
    return policy_Q[:-1]


@jax.jit
def compute_policy_matrix(policy: jnp.ndarray):
    """
    上で定義した方策行列を計算します。方策についての内積が取りたいときに便利です。
    Args:
        policy (jnp.ndarray): (HxSxA)の行列

    Returns:
        policy_matrix (jnp.ndarray): (HxSxSA)の行列
    """
    H, S, A = policy.shape
    PI = policy.reshape(H, 1, S, A)
    PI = jnp.tile(PI, (1, S, 1, 1))
    eyes = jnp.tile(jnp.eye(S).reshape(1, S, S, 1), (H, 1, 1, 1))
    PI = (eyes * PI).reshape(H, S, S*A)
    return PI


@jax.jit
def compute_policy_visit(mdp: MDP, policy: jnp.ndarray, init_dist: jnp.ndarray):
    """MDPと方策について、訪問頻度を動的計画法で計算します。
    Args:
        mdp (MDP)
        policy (jnp.ndarray): (HxSxA)の行列
        init_dist (jnp.ndarray): (S) 初期状態の分布

    Returns:
        visit (jnp.ndarray): (HxSxA)のベクトル
    """
    H, S, A = policy.shape
    Pi = compute_policy_matrix(policy)
    P = mdp.P.reshape(H, S*A, S)

    def body_fn(h, visit):
        next_visit = visit[h] @ P[h] @ Pi[h+1]
        visit = visit.at[h+1].set(next_visit)
        return visit
    
    visit = jnp.zeros((H+1, S*A))
    visit = visit.at[0].set((init_dist @ Pi[0]))
    visit = jax.lax.fori_loop(0, mdp.H, body_fn, visit)
    visit = visit[:-1].reshape(H, S, A)
    return visit


# 動的計画法による最適価値関数
optimal_Q_DP = compute_optimal_Q(mdp)
optimal_V_DP = optimal_Q_DP.max(axis=-1)
optimal_policy = compute_greedy_policy(optimal_Q_DP)
optimal_policy_Q_DP = compute_policy_Q(mdp, optimal_policy)
mdp = mdp._replace(optimal_Q=optimal_Q_DP)
print("最適価値関数と最適方策の価値関数の差", jnp.abs(optimal_Q_DP - optimal_policy_Q_DP).max())

# 訪問頻度によるリターンの計算
policy_visit = compute_policy_visit(mdp, optimal_policy, mdp.mu)
np.testing.assert_allclose(policy_visit.sum(axis=(1, 2)), 1.0, atol=1e-6)
np.testing.assert_allclose(policy_visit[0].sum(axis=-1), mdp.mu, atol=1e-6)
for h in range(H):
    return_by_visit = (policy_visit * mdp.rew)[h:].sum()
    return_by_DP = (optimal_Q_DP[h] * policy_visit[h]).sum()
    print(f"{h}ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差", np.abs(return_by_visit - return_by_DP))

最適価値関数と最適方策の価値関数の差 0.0
0ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 2.3841858e-07
1ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 2.3841858e-07
2ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 4.7683716e-07
3ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 1.1920929e-07
4ステップ目の訪問頻度によるリターンと動的計画法によるリターンの差 0.0


## 強化学習用

**実装した関数**

有限ホライゾンは本質的にリセット機能がついているので、あんまりGenerative modelの仮定がありません。

* ``sample_next_state``: ステップ・状態・行動の集合$D$のそれぞれについて次状態を$N$個返します
* ``collect_samples_eps_greedy``: $q\in \mathbb{R}^{H\times S\times A}$のε-貪欲方策で$H$回インタラクションしてサンプルを集めます

In [6]:
from jax.random import PRNGKey


@partial(jax.jit, static_argnames=("N",))
def sample_next_state(mdp: MDP, N: int, key: PRNGKey, D: jnp.array):
    """ 遷移行列Pに従って次の状態をN個サンプルします
    Args:
        mdp (MDP)
        N (int): サンプルする個数
        key (PRNGKey)
        D (np.ndarray): 状態行動対の集合 [(h1, s1, a1), (h1, s2, a2), ...]

    Returns:
        new_key (PRNGKey)
        next_s_set (np.ndarray): (len(D) x N) の次状態の集合
        count_HSAS (jnp.ndarray): 各(ステップ, 状態, 行動, 次状態)のペアの出現回数を格納した(H x S x A x S) の行列
    """
    new_key, key = jax.random.split(key)
    keys = jax.random.split(key, num=len(D))

    @jax.vmap
    def choice(key, hsa):
        return jax.random.choice(key, mdp.S_set, shape=(N,), p=P[hsa[0], hsa[1], hsa[2]])

    next_s = choice(keys, D)

    # 集めたサンプルについて、(h, s, a, ns)が何個出たかカウントします。
    H, S, A, S = mdp.P.shape
    count_HSAS = jnp.zeros((H*S*A, S))
    count_D_next_S = jax.vmap(lambda next_s: jnp.bincount(next_s, minlength=S, length=S))(next_s)
    D_ravel = jnp.ravel_multi_index(D.T, (H, S, A), mode="wrap")
    count_HSAS = count_HSAS.at[D_ravel].add(count_D_next_S)
    count_HSAS = count_HSAS.reshape(H, S, A, S)
    return new_key, next_s, count_HSAS


key = jax.random.PRNGKey(0)
N = 20000
D = jnp.array([(0, 1, 2), (1, 2, 1), (0, 0, 0), (4, 3, 1)])
key, next_states, count_HSAS = sample_next_state(mdp, N, key, D)

# next_statesによるPの推定
assert next_states.shape == (len(D), N)

for i, d in enumerate(D):
    h, s, a = d
    P0_approx1 = jnp.bincount(next_states[i], minlength=S) / N
    np.testing.assert_allclose(P0_approx1, mdp.P[h, s, a], atol=1e-2)

    # count_HSASによるPの推定
    P0_approx2 = count_HSAS[h, s, a] / N
    assert np.all(P0_approx1 == P0_approx2)

In [7]:
@jax.jit
def collect_samples_eps_greedy(mdp: MDP, key: PRNGKey, q: jnp.array, init_s: int, epsilon: float=0.0):
    """ エピソードの開始から終了まで、MDPとインタラクションしてサンプルをH個集めます。qのε-貪欲方策に従って動きます。
    Args:
        mdp (MDP)
        H (int): ホライゾン
        key (PRNGKey)
        q (jnp.ndarray): 行動価値関数
        init_s (int): 初期状態
        epsilon (float): ε-貪欲のパラメータ

    Returns:
        new_key (PRNGKey)
        sars (jnp.ndarray): (状態, 行動, 報酬, 次状態) x H の軌跡
        count_HSAS (jnp.ndarray): 各(ステップ, 状態, 行動, 次状態)のペアの出現回数を格納した(H x S x A x S) の行列
    """
    H, S, A, S = mdp.P.shape
    chex.assert_shape(q, (H, S, A))

    def body_fn(h, args):
        key, sars, s, count_HSAS = args

        # ε-貪欲方策を実行します
        a = q[h, s].argmax()
        key, key1, key2 = jax.random.split(key, num=3)
        random_a = jax.random.choice(key1, A)
        a = jnp.where(jax.random.uniform(key2) > epsilon, a, random_a)
        
        # 次状態をサンプルします
        key, key1 = jax.random.split(key)
        next_s = jax.random.choice(key1, mdp.S_set, p=P[h, s, a])

        # 集めたデータを記録します
        r = mdp.rew[h, s, a]
        sars = sars.at[h].set((s, a, r, next_s))
        count_HSAS = count_HSAS.at[h, s, a, next_s].add(1)
        return key, sars, next_s, count_HSAS

    sars = jnp.zeros((H, 4))
    count_HSAS = jnp.zeros((H, S, A, S))
    args = key, sars, init_s, count_HSAS
    key, sars, _, count_HSAS = jax.lax.fori_loop(0, H, body_fn, args)
    return key, sars, count_HSAS

key, sars, count_HSAS = collect_samples_eps_greedy(mdp, key, mdp.optimal_Q, 0, epsilon=1.0)
assert sars.shape == (mdp.H, 4)
assert count_HSAS.sum() == mdp.H
np.testing.assert_allclose(count_HSAS.sum(axis=(1, 2, 3)), 1.0)